In [39]:
import pandas as pd
import numpy as np
import requests as r

## Read the JSON file that you saved in ex02

- one of the columns has the float type, so let us define the format of it in pandas using pd.options.display.float_format: floats should be displayed with two decimals

- there are values missing from the Model, do not do anything with them

In [40]:
df = pd.read_json("../data/auto.json")
df.head()

,CarNumber,Refund,Fines,Make,Model
0,Y163O8161RUS,2,3200.00,Ford,Focus
1,E432XX77RUS,1,6500.00,Toyota,Camry
2,7184TT36RUS,1,2100.00,Ford,Focus
3,X582HE161RUS,2,2000.00,Ford,Focus
4,92918M178RUS,1,5700.00,Ford,Focus


In [41]:
pd.options.display.float_format = "{:.2f}".format
df.describe()

,Refund,Fines
count,725.00,725.00
mean,1.52,8594.59
std,0.50,16300.60
min,1.00,100.00
25%,1.00,1300.00
50%,2.00,4000.00
75%,2.00,8594.59
max,2.00,180000.00


## Enrich the dataframe using a sample from that dataframe

- create a sample with 200 new observations with random_state = 21
  - the sample should not have new combinations of the car number, make and model, so the whole dataset will be consistent in these terms
  - there are no restrictions on the refund and fines, you can take any value from these columns at random and use it towards any car number
- concatenate the sample with the initial dataframe to a new dataframe concat_rows

In [42]:
np.random.seed(21)

cars = df[['CarNumber', 'Make', 'Model']]
cars = cars.sample(n=200, replace=True, random_state=21)

cars['Fines'] = np.random.choice(df['Fines'], size=200)
cars['Refund'] = np.random.choice(df['Refund'], size=200)

cars

,CarNumber,Make,Model,Fines,Refund
207,Y351O8197RUS,Ford,Focus,1500.00,1
48,H917TC36RUS,Ford,Focus,4000.00,2
368,C589EY154RUS,Ford,Focus,4500.00,2
120,K846YE77RUS,Volkswagen,Passat,2000.00,2
419,X4108H125RUS,Ford,Focus,1300.00,2
...,...,...,...,...,...
587,M942OT152RUS,Ford,Focus,2000.00,1
595,Y187O8161RUS,Ford,Focus,400.00,2
365,7064C8197RUS,Volkswagen,Passat,12800.00,1
474,8437XX154RUS,Ford,Focus,800.00,2


In [43]:
concat_rows = pd.concat([df, cars])
concat_rows.reset_index(drop=True, inplace=True)

concat_rows

,CarNumber,Refund,Fines,Make,Model
0,Y163O8161RUS,2,3200.00,Ford,Focus
1,E432XX77RUS,1,6500.00,Toyota,Camry
2,7184TT36RUS,1,2100.00,Ford,Focus
3,X582HE161RUS,2,2000.00,Ford,Focus
4,92918M178RUS,1,5700.00,Ford,Focus
...,...,...,...,...,...
920,M942OT152RUS,1,2000.00,Ford,Focus
921,Y187O8161RUS,2,400.00,Ford,Focus
922,7064C8197RUS,1,12800.00,Volkswagen,Passat
923,8437XX154RUS,2,800.00,Ford,Focus


## Enrich the dataframe concat_rows by a new column with the data generated

- create a series with the name Year using random integers from 1980 to 2019
- use np.random.seed(21) before generating the years
- concatenate the series with the dataframe and name it fines

In [44]:
np.random.seed(21)

year = pd.Series(np.random.randint(1980, 2019, size=len(concat_rows)), name='Year')
fines = pd.concat([concat_rows, year], axis=1)

fines

,CarNumber,Refund,Fines,Make,Model,Year
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989
1,E432XX77RUS,1,6500.00,Toyota,Camry,1995
2,7184TT36RUS,1,2100.00,Ford,Focus,1984
3,X582HE161RUS,2,2000.00,Ford,Focus,2015
4,92918M178RUS,1,5700.00,Ford,Focus,2014
...,...,...,...,...,...,...
920,M942OT152RUS,1,2000.00,Ford,Focus,1996
921,Y187O8161RUS,2,400.00,Ford,Focus,2002
922,7064C8197RUS,1,12800.00,Volkswagen,Passat,1996
923,8437XX154RUS,2,800.00,Ford,Focus,2012


## Enrich the dataframe with the data from another dataframe

- create a new dataframe with the car numbers and their owners
  - get the most popular surnames (you can find the file surname.json in the attachments) in the US

In [45]:
surname_df = pd.read_json("../data/surname.json")
surname_df.columns = surname_df.iloc[0]
surname_df.drop(0, inplace=True)
surname_df.head()


,NAME,COUNT,RANK
1,ADAMS,427865,42
2,ALLEN,482607,33
3,ALVAREZ,233983,92
4,ANDERSON,784404,15
5,BAILEY,277845,72


- 
  - create a new series with the surnames (they should not have special characters like commas, brackets, etc.) from the data you gathered, the count should be equal to the number of unique car numbers using the sample (use random_state = 21)

In [46]:
surname_df = surname_df.loc[surname_df['NAME'].str.isalpha() == True]
surname_samples = surname_df.sample(
    fines['CarNumber'].unique().shape[0],
    random_state=21,
    replace=True)['NAME']
surname_samples
# surname_df

74    RICHARDSON
80          ROSS
57        MORGAN
5         BAILEY
49         LOPEZ
         ...    
10      CAMPBELL
32          HALL
6          BAKER
21          DIAZ
57        MORGAN
Name: NAME, Length: 531, dtype: object

- 
  - create the dataframe owners with 2 columns: CarNumber and SURNAME

In [47]:
owners = pd.DataFrame(
    zip(fines['CarNumber'].unique(), surname_samples),
    columns=['CarNumber', 'SURNAME']
)
owners

,CarNumber,SURNAME
0,Y163O8161RUS,RICHARDSON
1,E432XX77RUS,ROSS
2,7184TT36RUS,MORGAN
3,X582HE161RUS,BAILEY
4,92918M178RUS,LOPEZ
...,...,...
526,O136HO197RUS,CAMPBELL
527,O22097197RUS,HALL
528,M0309X197RUS,BAKER
529,O673E8197RUS,DIAZ


- append 5 more observations to the fines dataframe (come up with your own ideas of CarNumber, etc.)

In [48]:
my_data = pd.DataFrame({'CarNumber': ['R078TX178RUS', 'C718MC178RUS', 'K361KA178RUS', 'O432AB178RUS', 'X023HA178RUS'],
                        'Refund': [1, 2, 3, 4, 5],
                        'Fines': [2345.00, 5678.00, 8901.00, 1234.00, 2161.00],
                        'Make': ['McLaren', 'BMW', 'Lotus', 'Porsche', 'Volvo'],
                        'Model': ['P1', 'M760i', 'Esprit', '911 Targa', 'S90',],
                        'Year': [1999, 2021, 1987, 2024, 2017]})

fines = pd.concat([fines, my_data], ignore_index=True)
fines.tail()

,CarNumber,Refund,Fines,Make,Model,Year
925,R078TX178RUS,1,2345.00,McLaren,P1,1999
926,C718MC178RUS,2,5678.00,BMW,M760i,2021
927,K361KA178RUS,3,8901.00,Lotus,Esprit,1987
928,O432AB178RUS,4,1234.00,Porsche,911 Targa,2024
929,X023HA178RUS,5,2161.00,Volvo,S90,2017


- delete the dataframe last 20 observations from the owners and add 3 new observations (they are not the same as those you add to the fines dataframe)

In [49]:
owners.drop(owners.tail(20).index, inplace=True)
owners

,CarNumber,SURNAME
0,Y163O8161RUS,RICHARDSON
1,E432XX77RUS,ROSS
2,7184TT36RUS,MORGAN
3,X582HE161RUS,BAILEY
4,92918M178RUS,LOPEZ
...,...,...
506,T914CT197RUS,HERNANDEZ
507,E41977152RUS,BAKER
508,9464EX178RUS,MARTIN
509,O50197197RUS,WRIGHT


In [51]:
my_data = pd.DataFrame({
    'CarNumber': ['E778MX05RUS', 'H973EE05RUS', 'M558MB05RUS'],
    'SURNAME': ['PUTIN', 'PUTIN', 'PUTIN']
})
owners = pd.concat([owners, my_data], ignore_index=True)

owners

,CarNumber,SURNAME
0,Y163O8161RUS,RICHARDSON
1,E432XX77RUS,ROSS
2,7184TT36RUS,MORGAN
3,X582HE161RUS,BAILEY
4,92918M178RUS,LOPEZ
...,...,...
509,O50197197RUS,WRIGHT
510,7608EE777RUS,HILL
511,E778MX05RUS,PUTIN
512,H973EE05RUS,PUTIN


- join both dataframes:
  - the new dataframe should have only the car numbers that exist in both dataframes
  - the new dataframe should have all the car numbers that exist in both dataframes
  - the new dataframe should have only the car numbers from the fines dataframe
  - the new dataframe should have only the car numbers from the owners dataframe

In [52]:
pd.merge(fines, owners, how='inner', left_on='CarNumber', right_on='CarNumber')

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989,RICHARDSON
1,E432XX77RUS,1,6500.00,Toyota,Camry,1995,ROSS
2,7184TT36RUS,1,2100.00,Ford,Focus,1984,MORGAN
3,X582HE161RUS,2,2000.00,Ford,Focus,2015,BAILEY
4,92918M178RUS,1,5700.00,Ford,Focus,2014,LOPEZ
...,...,...,...,...,...,...,...
898,M942OT152RUS,1,2000.00,Ford,Focus,1996,ALVAREZ
899,Y187O8161RUS,2,400.00,Ford,Focus,2002,COOK
900,7064C8197RUS,1,12800.00,Volkswagen,Passat,1996,DAVIS
901,8437XX154RUS,2,800.00,Ford,Focus,2012,HALL


In [53]:
pd.merge(fines, owners, how='outer', left_on='CarNumber', right_on='CarNumber')

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,704687163RUS,2.00,1400.00,Ford,Focus,2014.00,ADAMS
1,704787163RUS,2.00,2800.00,Ford,Focus,2005.00,MORGAN
2,704987163RUS,2.00,8594.59,Ford,Focus,2014.00,MITCHELL
3,705287163RUS,2.00,2000.00,Ford,Focus,1990.00,GOMEZ
4,705387163RUS,2.00,700.00,Ford,Focus,2005.00,STEWART
...,...,...,...,...,...,...,...
928,Y969O8197RUS,2.00,7800.00,Ford,Focus,1987.00,LOPEZ
929,Y973O8197RUS,2.00,8594.59,Ford,Focus,2005.00,YOUNG
930,Y973O8197RUS,1.00,34800.00,Ford,Focus,2013.00,YOUNG
931,Y973O8197RUS,1.00,69600.00,Ford,Focus,1989.00,YOUNG


In [54]:
pd.merge(fines, owners, how='left', left_on='CarNumber', right_on='CarNumber')

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989,RICHARDSON
1,E432XX77RUS,1,6500.00,Toyota,Camry,1995,ROSS
2,7184TT36RUS,1,2100.00,Ford,Focus,1984,MORGAN
3,X582HE161RUS,2,2000.00,Ford,Focus,2015,BAILEY
4,92918M178RUS,1,5700.00,Ford,Focus,2014,LOPEZ
...,...,...,...,...,...,...,...
925,R078TX178RUS,1,2345.00,McLaren,P1,1999,NaN
926,C718MC178RUS,2,5678.00,BMW,M760i,2021,NaN
927,K361KA178RUS,3,8901.00,Lotus,Esprit,1987,NaN
928,O432AB178RUS,4,1234.00,Porsche,911 Targa,2024,NaN


In [55]:
pd.merge(fines, owners, how='right', left_on='CarNumber', right_on='CarNumber')

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,Y163O8161RUS,2.00,3200.00,Ford,Focus,1989.00,RICHARDSON
1,Y163O8161RUS,2.00,1600.00,Ford,Focus,1999.00,RICHARDSON
2,Y163O8161RUS,2.00,3200.00,Ford,Focus,1988.00,RICHARDSON
3,Y163O8161RUS,2.00,3200.00,Ford,Focus,1996.00,RICHARDSON
4,Y163O8161RUS,2.00,1600.00,Ford,Focus,2013.00,RICHARDSON
...,...,...,...,...,...,...,...
901,O50197197RUS,2.00,7800.00,Ford,Focus,1986.00,WRIGHT
902,7608EE777RUS,1.00,4000.00,Skoda,Octavia,2013.00,HILL
903,E778MX05RUS,NaN,NaN,NaN,NaN,NaN,PUTIN
904,H973EE05RUS,NaN,NaN,NaN,NaN,NaN,PUTIN


## Create a pivot table from the fines dataframe, it should look like this (the values are the sums of the fines), but with all the years (the values may be different for you):

In [58]:
fines
pd.pivot_table(
    fines,
    index=['Make', 'Model'],
    values='Fines',
    columns='Year'
)

Year                     1980     1981     1982     1983    1984      1985  \
Make       Model                                                             
BMW        M760i          NaN      NaN      NaN      NaN     NaN       NaN   
Ford       Focus      4718.41 15369.73  6494.45 11785.33 7031.25  10434.41   
           Mondeo         NaN      NaN 46200.00      NaN     NaN       NaN   
Lotus      Esprit         NaN      NaN      NaN      NaN     NaN       NaN   
McLaren    P1             NaN      NaN      NaN      NaN     NaN       NaN   
Porsche    911 Targa      NaN      NaN      NaN      NaN     NaN       NaN   
Skoda      Octavia    8594.59  1900.00  3631.53      NaN  300.00   8594.59   
Toyota     Camry     12000.00      NaN  7200.00      NaN 1000.00       NaN   
           Corolla        NaN  6800.00      NaN  6400.00     NaN       NaN   
Volkswagen Golf      20800.00  8594.59  5000.00   200.00  200.00 168000.00   
           Jetta          NaN  1000.00      NaN      NaN     NaN   9000.00   
           Passat      900.00  3000.00      NaN   550.00 8594.59       NaN   
           Touareg        NaN      NaN      NaN      NaN     NaN       NaN   
Volvo      S90            NaN      NaN      NaN      NaN     NaN       NaN   

Year                     1986     1987     1988    1989  ...    2011    2012  \
Make       Model                                         ...                   
BMW        M760i          NaN      NaN      NaN     NaN  ...     NaN     NaN   
Ford       Focus      4784.62  8329.73  5514.86 7918.79  ... 5618.79 6885.20   
           Mondeo         NaN      NaN      NaN     NaN  ...     NaN     NaN   
Lotus      Esprit         NaN  8901.00      NaN     NaN  ...     NaN     NaN   
McLaren    P1             NaN      NaN      NaN     NaN  ...     NaN     NaN   
Porsche    911 Targa      NaN      NaN      NaN     NaN  ...     NaN     NaN   
Skoda      Octavia        NaN  2000.00  1700.00 4547.29  ... 3000.00  850.00   
Toyota     Camry     19800.00      NaN      NaN  800.00  ...     NaN 7500.00   
           Corolla        NaN 23433.33      NaN 7800.00  ...     NaN     NaN   
Volkswagen Golf           NaN      NaN      NaN 1300.00  ...     NaN     NaN   
           Jetta          NaN      NaN 46000.00     NaN  ...     NaN     NaN   
           Passat     8000.00  2000.00  6729.72     NaN  ...     NaN     NaN   
           Touareg        NaN      NaN      NaN     NaN  ...     NaN     NaN   
Volvo      S90            NaN      NaN      NaN     NaN  ...     NaN     NaN   

Year                     2013    2014    2015     2016    2017     2018  \
Make       Model                                                          
BMW        M760i          NaN     NaN     NaN      NaN     NaN      NaN   
Ford       Focus     10128.18 6509.67 8036.36  7746.10 4736.12  5206.25   
           Mondeo    20550.00     NaN     NaN      NaN 8600.00      NaN   
Lotus      Esprit         NaN     NaN     NaN      NaN     NaN      NaN   
McLaren    P1             NaN     NaN     NaN      NaN     NaN      NaN   
Porsche    911 Targa      NaN     NaN     NaN      NaN     NaN      NaN   
Skoda      Octavia    5098.65     NaN 5464.86 11900.00 2400.00 53800.00   
Toyota     Camry          NaN     NaN 7500.00      NaN     NaN  6500.00   
           Corolla        NaN 6800.00     NaN      NaN     NaN      NaN   
Volkswagen Golf           NaN 6950.00     NaN   300.00     NaN      NaN   
           Jetta          NaN     NaN     NaN      NaN     NaN      NaN   
           Passat     1600.00     NaN     NaN      NaN     NaN      NaN   
           Touareg        NaN     NaN     NaN      NaN     NaN      NaN   
Volvo      S90            NaN     NaN     NaN      NaN 2161.00      NaN   

Year                    2021    2024  
Make       Model                      
BMW        M760i     5678.00     NaN  
Ford       Focus         NaN     NaN  
           Mondeo        NaN     NaN  
Lotus      Esprit        NaN     NaN  
McLaren    P1            NaN     Na

In [59]:
fines.to_csv("../data/fines.csv", index=False)
owners.to_csv("../data/owners.csv", index=False)